In [1]:
# nse scratchpad initiation
from helper import *
from nse_func import *

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

In [2]:
ib =  get_connected('nse', 'live')

In [3]:
# get the remaining quantities
dfrq = remqty_nse(ib)

dft = pd.read_pickle(fspath+'targets.pickle')

dft = dft.assign(remqty=dft.symbol.map(dfrq.remqty.to_dict()))   # remaining quantities
dft = dft[dft.remqty > 0].reset_index(drop=True) # remove blacklisted (remqty <= 0)

In [4]:
pmask = (dft.right == 'P') & (dft.undPrice-dft.strike-dft.expPrice > dft.Fall)

df_prisky = pd.merge(dft[pmask][['symbol', 'strike', 'right', 'dte', 'undPrice', 'optPrice', 'expPrice', 'Fall', 'Rise', 'expRom']], 
         pd.DataFrame((dft[pmask].undPrice-dft[pmask].strike-dft[pmask].expPrice)), on=dft[pmask].index).drop('key_0', 1)

cmask = (dft.right == 'C') & (dft.strike-dft.undPrice-dft.expPrice > dft.Rise)
df_crisky = pd.merge(dft[cmask][['symbol', 'strike', 'right', 'dte', 'undPrice', 'optPrice', 'expPrice', 'Fall', 'Rise', 'expRom']], 
         pd.DataFrame((dft[cmask].strike-dft[cmask].undPrice-dft[cmask].expPrice)), on=dft[cmask].index).drop('key_0', 1)

df_risky = pd.concat([df_prisky, df_crisky]).reset_index(drop=True)

df_risky = df_risky.rename(columns={0: 'FallRise'})

In [5]:
df_risky

,symbol,strike,right,dte,undPrice,optPrice,expPrice,Fall,Rise,expRom,FallRise
0,BAJAJ-AUT,2350.0,P,17,2966.20,1.70,25.20,589.450,539.90,1.000872,591.00
1,BAJAJ-AUT,2300.0,P,17,2966.20,1.50,25.20,589.450,539.90,1.000872,641.00
2,ICICIPRUL,260.0,P,17,352.10,0.30,2.95,82.150,90.10,0.995613,89.15
3,ICICIPRUL,250.0,P,17,352.10,0.20,2.95,82.150,90.10,0.995613,99.15
4,KOTAKBANK,980.0,P,17,1398.15,0.25,11.85,205.550,222.20,1.000088,406.30
5,KOTAKBANK,960.0,P,17,1398.15,0.20,11.85,205.550,222.20,1.000088,426.30
6,NIFTY50,9800.0,P,17,11278.90,19.95,63.50,1325.950,842.90,0.999970,1415.40
7,PVR,1050.0,P,17,1715.40,2.60,14.65,361.050,279.10,1.001017,650.75
8,PVR,950.0,P,17,1715.40,1.30,14.65,361.050,279.10,1.001017,750.75
9,RBLBANK,420.0,P,17,675.00,0.10,5.70,133.950,123.45,1.002332,249.30


In [6]:
dynamic(ib)

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Peer closed connection
